<a href="https://colab.research.google.com/github/midhunjmes/midhun-26-02-24/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

hi


In [2]:
pip install presidio_analyzer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.1 MB/s eta 0:00:00


In [3]:
pip install presidio_anonymizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 14.5 MB/s eta 0:00:00


In [7]:
!pip install spacy


In [8]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

In [79]:
analyzer=AnalyzerEngine()
anonymizer=AnonymizerEngine()
text="Alwin's phone number is 1234567890 and email id is mid@gmail.com and I lives at pune,Ernakulam "
results= analyzer.analyze(text=text,entities=["EMAIL_ADDRESS","PHONE_NUMBER","PERSON","LOCATION","STREET_ADDRESS"],language="en")
#print(results)
result_text=anonymizer.anonymize(text,results)
print(result_text)


text: <PERSON>'s phone number is <PHONE_NUMBER> and email id is <EMAIL_ADDRESS> and I lives at <LOCATION>,<LOCATION> 
items:
[
    {'start': 100, 'end': 110, 'entity_type': 'LOCATION', 'text': '<LOCATION>', 'operator': 'replace'},
    {'start': 89, 'end': 99, 'entity_type': 'LOCATION', 'text': '<LOCATION>', 'operator': 'replace'},
    {'start': 58, 'end': 73, 'entity_type': 'EMAIL_ADDRESS', 'text': '<EMAIL_ADDRESS>', 'operator': 'replace'},
    {'start': 27, 'end': 41, 'entity_type': 'PHONE_NUMBER', 'text': '<PHONE_NUMBER>', 'operator': 'replace'},
    {'start': 0, 'end': 8, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'}
]



In [60]:
result_text=anonymizer.anonymize(text,results)
print(result_text)

text: <PERSON>'s phone number is <PHONE_NUMBER> and email id is <EMAIL_ADDRESS> and my location is <LOCATION>
items:
[
    {'start': 93, 'end': 103, 'entity_type': 'LOCATION', 'text': '<LOCATION>', 'operator': 'replace'},
    {'start': 58, 'end': 73, 'entity_type': 'EMAIL_ADDRESS', 'text': '<EMAIL_ADDRESS>', 'operator': 'replace'},
    {'start': 27, 'end': 41, 'entity_type': 'PHONE_NUMBER', 'text': '<PHONE_NUMBER>', 'operator': 'replace'},
    {'start': 0, 'end': 8, 'entity_type': 'PERSON', 'text': '<PERSON>', 'operator': 'replace'}
]



In [80]:
import json

In [82]:
def anonymize_json(data):
    if isinstance(data, dict):
        return {key: anonymize_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [anonymize_json(item) for item in data]
    elif isinstance(data, str):
        # Detect sensitive data
        results = analyzer.analyze(text=data, entities=["PERSON", "EMAIL_ADDRESS", "PHONE_NUMBER", "LOCATION"], language="en")
        # Anonymize the detected entities
        return anonymizer.anonymize(text=data, analyzer_results=results).text if results else data
    else:
        return data

In [86]:

json_data = {
    "name": "Alice Johnson",
    "email": "alice@example.com",
    "phone": "+911234567890",
    "address": "123 Main St, kochi",
    "messages": [
        "Hello, this is Alice!",
        "My email is alice@example.com and I live in New York."
    ]
}
anonymized_data = anonymize_json(json_data)
print(json.dumps(anonymized_data, indent=2))

{
  "name": "<PERSON>",
  "email": "<EMAIL_ADDRESS>",
  "phone": "<PHONE_NUMBER>",
  "address": "123 <LOCATION>, <LOCATION>",
  "messages": [
    "Hello, this is <PERSON>!",
    "My email is <EMAIL_ADDRESS> and I live in <LOCATION>."
  ]
}
